In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40 kB 1.8 MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-py3-none-any.whl size=30114 sha256=9d4901b76f5faca5fada693ff88ee92b232021d88aae664923ef1f6f2c97ed18
  Stored in directory: /root/.cache/pip/wheels/7c/b5/ea/3e768502fa1032a6bfefcd0e7dd95e2110e026bf3c584940cd
  Created wheel for py-params: filename=py_params-0.9.7-py3-none-any.whl size=7302 sha256=30a8b0985f4d927afbd817f59233db1b041f71331f1eb83b0bf37010f6afb287
  Stored in directory: /root/.cache/pip/wheels/18/e2/80/244168a554fc8469897bb86b3ea7183b56ecc92f484e67b375
  Created wheel for params-flow: filename=params_flow-0.8.1-py3-none-any.whl size=16121 sha256=e629d1a58f37bca7b1b4d2ab6cfb5262a57cb5cedde02cdbf5042b5b728c3449
  Stored in directory: /root/.cache/pip/wheels/cc/70/6f/98e4cd9239940e8160866872af2be0ca038cdfbdc8113db2ad
Successfully built bert-for-tf2 py-params params-flow


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/extensive-pre-processing-for-bert/processed train.csv
/kaggle/input/extensive-pre-processing-for-bert/processed test.csv
/kaggle/input/extensive-pre-processing-for-bert/custom.css
/kaggle/input/extensive-pre-processing-for-bert/__notebook__.ipynb
/kaggle/input/extensive-pre-processing-for-bert/__results__.html
/kaggle/input/extensive-pre-processing-for-bert/__output__.json


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from tqdm import tqdm
# from keras.models import Sequential
# from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
# from keras.initializers import Constant
# from keras.optimizers import Adam
# from keras import regularizers
# import keras.backend as K

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
import tensorflow_hub as hub

try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import bert


In [4]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
#     y_true = K.argmax(y_true_in, axis=1)
#     y_pred = K.argmax(y_pred_in, axis=1)
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
train= pd.read_csv('../input/extensive-pre-processing-for-bert/processed train.csv')
train.head(5)

,id,keyword,location,text,target,processed_text
0,10517,wildfire,NaN,! Residents Return To Destroyed Homes As Washi...,1,! residents return to destroyed homes as washi...
1,7768,police,USA,$1 million bail for man accused of #shooting a...,1,$1 million bail for man accused of shooting at...
2,3990,devastation,"Denver, CO",$10M SETTLEMENT attained using our illustratio...,1,$10m settlement attained using our illustratio...
3,178,aftershock,United States,&gt;&gt; $15 Aftershock : Protect Yourself and...,0,$15 aftershock : protect yourself and profit i...
4,2657,crush,NaN,'@jorrynja: 6. @ your bf/gf/crush ??' @Ter_ell ??,1,' : 6. your bf/gf/crush ??' ??


In [6]:
train.loc[4,'processed_text']

"' : 6. your bf/gf/crush ??' ??"

In [7]:
test=pd.read_csv('../input/extensive-pre-processing-for-bert/processed test.csv')
test = test.set_index(test['id'])
test.head(5)

,id,keyword,location,text,processed_text
id,,,,,
0,0,NaN,NaN,Just happened a terrible car crash,just happened a terrible car crash
2,2,NaN,NaN,"Heard about #earthquake is different cities, s...","heard about earthquake is different cities, st..."
3,3,NaN,NaN,"there is a forest fire at spot pond, geese are...","there is a forest fire at spot pond, geese are..."
9,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting. spokane wildfires
11,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills 28 in china and taiwan


In [8]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
# module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
#https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1
#https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/1
bert_layer = hub.KerasLayer(module_url, trainable=True)


# BERT Encodings

In [9]:
"""BERT Methods Predefined"""
def bert_encode(texts, tokenizer, max_len=50):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


In [10]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

There are 6934 rows and 6 columns in train
There are 3263 rows and 5 columns in train


In [11]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocab_file, do_lower_case)

In [12]:
full_input = bert_encode(train.processed_text.values, tokenizer, max_len=50)
full_labels = train.target.values.copy()
full_labels_oe = to_categorical(full_labels, 2)

In [13]:
train_data, val_data, train_labels, val_labels = train_test_split(train.processed_text.values, train.target.values, test_size=0.15, random_state=10)
train_input = bert_encode(train_data, tokenizer, max_len=50)
val_input = bert_encode(val_data, tokenizer, max_len=50)
test_input = bert_encode(test.processed_text.values, tokenizer, max_len=50)

test_labels_oe = to_categorical(test_labels, 2)
val_labels_oe = to_categorical(val_labels, 2)

NameError: name 'test_labels' is not defined

In [14]:
learning_rate=9e-6
decay=9e-2
max_len=50

K.clear_session()

In [15]:
input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
clf_output = sequence_output[:, 0, :]
# clf_pooled = pooled_output[:, 0, :]

## Type1
# out = Dense(100, activation='relu', activity_regularizer=regularizers.l2(9e-5))(clf_output)
# out = Dense(100, activation='relu', activity_regularizer=regularizers.l2(9e-5))(out)
# out = Dense(100, activation='relu')(out)

## Type2
# out = Dropout(Dropout_num)(clf_output)
out = Dense(1024, activation='relu', activity_regularizer=regularizers.l2(9e-5))(clf_output)
out = Dense(1024, activation='relu', activity_regularizer=regularizers.l2(9e-5))(out)
out = Dense(1024, activation='relu')(out)

## Type3
# out = clf_output


out = Dense(1, activation='sigmoid')(out)


## Type4
# out = clf_output

In [16]:
sBERT = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
# sBERT.compile(SGD(lr=learning_rate, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])
sBERT.compile(Adam(lr=learning_rate, decay=decay), loss='binary_crossentropy', metrics=['accuracy',f1_m])
sBERT.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [17]:
# init_weights = sBERT.get_weights()

In [18]:
# sBERT.set_weights(init_weights)
# sBERT.layers[3].trainable = True
# sBERT.compile(Adam(lr=9e-6, decay=9e-2), loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
checkpoint1 = ModelCheckpoint('best_accuracy.h5',
                             monitor='val_f1_m',
                             save_best_only=True)
checkpoint2 = ModelCheckpoint('best_loss.h5',
                             monitor='val_loss',
                             save_best_only=True)


train_history = sBERT.fit(
    full_input, full_labels,
    epochs = 3,
    batch_size = 12
#     callbacks = [checkpoint1]
)

Train on 6934 samples
Epoch 1/3
6934/6934 [==============================] - 6690s 965ms/sample - loss: 0.5882 - accuracy: 0.7391 - f1_m: 0.5409
Epoch 2/3
6934/6934 [==============================] - 6709s 967ms/sample - loss: 0.5250 - accuracy: 0.7850 - f1_m: 0.6779
Epoch 3/3
6934/6934 [==============================] - 6713s 968ms/sample - loss: 0.5069 - accuracy: 0.7900 - f1_m: 0.6897


In [20]:
# # K.set_value(sBERT.optimizer.lr, 9e-7)
# train_history = sBERT.fit(
#     full_input, full_labels,
#     epochs = 3,
#     batch_size = 16
# #     callbacks = [checkpoint1]
# )

In [21]:
# sBERT.layers[3].trainable = False
# sBERT.compile(Adam(lr=5e-4, decay=5e-2), loss='binary_crossentropy', metrics=['accuracy',f1_m])

In [22]:
# # K.set_value(sBERT.optimizer.lr, 5e-3)
# sBERT.fit(
#     full_input, full_labels,
#     epochs = 5,
#     callbacks=[checkpoint1],
#     batch_size = 16
# )

In [23]:
# sBERT.load_weights('testacc.h5')

In [24]:
bert_encoder = Model(sBERT.inputs, sBERT.layers[4].output)
bert_encoder.summary()

# bert_encoder.layers[-3].set_weights(sBERT.layers[-4].get_weights())
# bert_encoder.layers[-2].set_weights(sBERT.layers[-3].get_weights())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [25]:
%%time
full_embed = bert_encoder.predict(full_input)
train_embed = bert_encoder.predict(train_input)
test_embed = bert_encoder.predict(test_input)

CPU times: user 4h 37min 29s, sys: 59.3 s, total: 4h 38min 28s
Wall time: 1h 12min 56s


# Pickle BERT Encodings for further use

In [26]:
import pickle
with open('Train BERT 1024d Embed', 'ab') as fo:     
    pickle.dump(full_embed, fo)    

with open('Test BERT 1024d Embed', 'ab') as fo:     
    pickle.dump(test_embed, fo)  

In [27]:
# with open('Train BERT 1024d Embed', 'rb') as fo:
#     tt = pickle.load(fo, encoding='latin1')

# Support Vector Machine

In [28]:
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV
from sklearn.svm import SVC

In [29]:
%%time
svc_model = SVC(gamma=0.01, kernel='rbf', C=3)
svc_model.fit(full_embed, full_labels)
# svc_model.fit(train_embed, train_labels)

CPU times: user 1min 20s, sys: 65.6 ms, total: 1min 20s
Wall time: 1min 20s


SVC(C=3, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [30]:
import xgboost as xgb

In [31]:
%%time

clf = xgb.XGBClassifier(max_depth=200, n_estimators=400, subsample=1, learning_rate=0.07, reg_lambda=0.1, reg_alpha=0.1,\
                       gamma=1)
clf.fit(full_embed, full_labels)
# clf.fit(train_embed, train_labels)

predictions = clf.predict(full_embed)
# predictions = clf.predict(train_embed)
print ("Training set f1_score :", np.round(f1_score(train_labels, predictions),5))

ValueError: Found input variables with inconsistent numbers of samples: [5893, 6934]

In [32]:
test_pred1 = clf.predict(test_embed).round().astype(int)
test_pred2 = svc_model.predict(test_embed).round().astype(int)
test_pred3 = sBERT.predict(test_input).round().astype(int)
print("XGBOOST: ", accuracy_score(test_labels, test_pred1), f1_score(test_labels, test_pred1))
print("SVC: ",accuracy_score(test_labels, test_pred2), f1_score(test_labels, test_pred2))
print("BERT: ",accuracy_score(test_labels, test_pred3), f1_score(test_labels, test_pred3))

NameError: name 'test_labels' is not defined

In [33]:
sub = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

sub['target'] = test_pred1
sub.to_csv('submission_xgboost.csv', index=False)

sub['target'] = test_pred2
sub.to_csv('submission_svc.csv', index=False)

sub['target'] = test_pred3
sub.to_csv('submission_bertnn.csv', index=False)